# Scrape UFCStats.com

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
def get_event_links():
    url = 'http://ufcstats.com/statistics/events/completed?page=all'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    event_links = []
    for a in soup.select('a.b-link.b-link_style_black'):
        href = a.get('href')
        if href and 'event-details' in href:
            event_links.append(href)
    
    return list(set(event_links))  # Remove duplicates

# Test it
event_urls = get_event_links()
print(f"Collected {len(event_urls)} event URLs.")

Collected 731 event URLs.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_event_links():
    """
    Scrape the page with all completed events and return a list of event URLs.
    """
    url = 'http://ufcstats.com/statistics/events/completed?page=all'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    event_links = []
    # The event links are found in anchor tags with class 'b-link b-link_style_black'
    for a in soup.select('a.b-link.b-link_style_black'):
        href = a.get('href')
        if href and 'event-details' in href:
            event_links.append(href)
    return list(set(event_links))  # Remove duplicates if any

def get_event_details(soup):
    """
    Given the BeautifulSoup object of an event page, extract event-level details.
    """
    details = {}
    # Assume event date and location are in <li> tags with class 'b-list__box-list-item'
    items = soup.find_all('li', class_='b-list__box-list-item')
    for item in items:
        text = item.get_text(strip=True)
        if 'Date:' in text:
            details['Event Date'] = text.replace('Date:', '').strip()
        elif 'Location:' in text:
            details['Location'] = text.replace('Location:', '').strip()
    return details

def parse_table_dual_rows(table):
    """
    Parses tables where each <td> has two <p> tags representing two fighters.
    Returns a dict of stat_name: (fighter1_val, fighter2_val)
    """
    headers = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]
    rows = table.find("tbody").find_all("tr")[0].find_all("td")

    data = {"fighter_1": {}, "fighter_2": {}}
    for i, header in enumerate(headers[1:], 1):
        values = rows[i].find_all("p")
        if len(values) == 2:
            data["fighter_1"][header] = values[0].get_text(strip=True)
            data["fighter_2"][header] = values[1].get_text(strip=True)
    return data


def get_fight_stats(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Extract fighter names
    fighter_tags = soup.select(".b-fight-details__person-name a")
    if len(fighter_tags) < 2:
        return None
    fighter_1 = fighter_tags[0].get_text(strip=True)
    fighter_2 = fighter_tags[1].get_text(strip=True)

    fight_data = {
        "fighter_1_name": fighter_1,
        "fighter_2_name": fighter_2,
    }

    # Extract metadata: method, round, time, referee, etc.
    metadata_section = soup.select_one(".b-fight-details__content")
    if metadata_section:
        for item in metadata_section.select(".b-fight-details__text-item"):
            label = item.select_one(".b-fight-details__label")
            if label:
                key = label.get_text(strip=True).replace(":", "").lower()
                value = item.get_text(strip=True).replace(label.get_text(strip=True), "").strip()
                fight_data[key] = value

    # Extract fight stats summary table
    tables = soup.find_all("table", class_="b-fight-details__table")
    if len(tables) > 0:
        fight_data["overall_stats"] = parse_table_dual_rows(tables[0])

    # Extract per-round summary stats
    if len(tables) > 1:
        fight_data["per_round_stats"] = {}
        round_tables = soup.select("table.b-fight-details__table.js-fight-table")
        for i, round_table in enumerate(round_tables):
            fight_data["per_round_stats"][f"round_{i+1}"] = parse_table_dual_rows(round_table)
            

    # Extract significant strikes breakdown (overall)
    if len(tables) > 2:
        fight_data["sig_strike_breakdown"] = parse_table_dual_rows(tables[2])

    return fight_data

def get_fights_from_event(event_url):
    """
    Given an event URL, scrape the event page to extract:
      - Basic fight details (fighter names, winner, weight class, method, round, time)
      - Event details (date and location)
      - Detailed fighter statistics from the fight detail page
    """
    fights = []
    try:
        response = requests.get(event_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Get event-level details (date and location)
        event_details = get_event_details(soup)
        
        # Locate the fight table on the event page.
        table = soup.find('table', class_='b-fight-details__table')
        if not table:
            return fights

        rows = table.find_all('tr')[1:]  # Skip header row
        for row in rows:
            cols = row.find_all('td')
            # Verify there are enough columns before indexing
            if len(cols) < 10:
                continue

            # Basic fight details (you may need to adjust the indices based on the site’s structure)
            fight = {
                'Fighter A': cols[1].get_text(strip=True),
                'Fighter B': cols[2].get_text(strip=True),
                'Winner': cols[0].get_text(strip=True),
                'Weight Class': cols[6].get_text(strip=True),
                'Method': cols[7].get_text(strip=True),
                'Round': cols[8].get_text(strip=True),
                'Time': cols[9].get_text(strip=True),
                'Event Date': event_details.get('Event Date', None),
                'Location': event_details.get('Location', None)
            }
            
            # Extract the fight detail URL – typically found in an <a> tag in the row.
            fight_link_tag = row.find('a')
            #print(f'fight_link_tag:{fight_link_tag}')
            fight_detail_url = fight_link_tag['href'] if fight_link_tag else None
            #print(f'fight_detail_url:{fight_detail_url}')
            if fight_detail_url:
                fight_stats = get_fight_stats(fight_detail_url)
                fight.update(fight_stats)
                time.sleep(1)  # Delay between fight detail requests
            
            fights.append(fight)
    except Exception as e:
        print(f"Error processing event {event_url}: {e}")
    return fights

def scrape_ufc_data(limit_events=None):
    """
    Scrape data for all events (or limit to a set number) and compile fight details.
    """
    all_fights = []
    event_urls = get_event_links()
    print(f"Found {len(event_urls)} events.")
    if limit_events:
        event_urls = event_urls[:limit_events]
    
    for i, event_url in enumerate(event_urls, start=1):
        print(f"Processing event {i}/{len(event_urls)}: {event_url}")
        fights = get_fights_from_event(event_url)
        all_fights.extend(fights)
        time.sleep(2)  # Delay between event requests
        
    return all_fights

if __name__ == "__main__":
    # For testing purposes, limit to the first 10 events. Remove or adjust 'limit_events' for full scraping.
    fights_data = scrape_ufc_data(limit_events=1)
    df = pd.DataFrame(fights_data)
    df.to_csv('ufc_fight_history_extended.csv', index=False)
    print("Saved extended UFC fight data to 'ufc_fight_history_extended.csv'")

Found 725 events.
Processing event 1/1: http://ufcstats.com/event-details/bd4389b71fdc0ce2
Saved extended UFC fight data to 'ufc_fight_history_extended.csv'


In [8]:
import ast

df = pd.read_csv(r'ufc_fight_history_extended.csv')

df['overall_stats'] = df['overall_stats'].apply(ast.literal_eval)
df['per_round_stats'] = df['per_round_stats'].apply(ast.literal_eval)

rows = []

for idx, row in df.iterrows():
    fight_id = idx  # or df has a unique ID
    for role in ['fighter_1', 'fighter_2']:
        fighter_name = row[f"{role}_name"]
        stats = row['overall_stats'].get(role, {})
        
        flat = {
            'fight_id': fight_id,
            'fighter_name': fighter_name,
            **stats
        }
        rows.append(flat)

overall_stats_df = pd.DataFrame(rows)

In [9]:
overall_stats_df

,fight_id,fighter_name,KD,Sig. str.,Sig. str. %,Total str.,Td %,Sub. att,Rev.,Ctrl
0,0,Demetrious Johnson,0,7 of 17,41%,9 of 19,66%,0,0,1:20
1,0,Tim Elliott,0,12 of 22,54%,23 of 35,100%,2,0,2:20
2,1,Joseph Benavidez,0,22 of 69,31%,22 of 69,---,0,0,0:01
3,1,Henry Cejudo,1,21 of 66,31%,21 of 66,33%,0,0,0:06
4,2,Jake Ellenberger,0,11 of 31,35%,11 of 31,0%,0,0,0:03
5,2,Jorge Masvidal,0,44 of 63,69%,50 of 70,---,0,0,0:38
6,3,Ion Cutelaba,0,24 of 53,45%,25 of 54,100%,0,0,2:20
7,3,Jared Cannonier,0,11 of 17,64%,16 of 22,---,0,0,0:35
8,4,Sara McMann,0,12 of 30,40%,21 of 39,100%,0,0,0:00
9,4,Alexis Davis,0,17 of 21,80%,51 of 58,---,1,0,3:49
